In [214]:
%matplotlib notebook
from matplotlib import pyplot as plt
import json
from __future__ import print_function
from collections import Counter
from datetime import datetime
import pandas as pd
import numpy as np

In [241]:
# Sentiment Intensity Analyzer Using Vader
from __future__ import print_function
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize
sid = SentimentIntensityAnalyzer()

with open ("test2.json") as f:
    #comma delimited json requires read_json method to set lines parameter to true
    df = pd.read_json(json.loads(json.dumps(f.read())), lines=True)
    #list unique business IDS and user ids to construct full dataframe
    users = df.user_id.unique()
    businesses = df.business_id.unique()
    utility_matrix = pd.DataFrame(index=users, columns=businesses)
    #print(df.columns)
    #print(utility_matrix)
    for index, row in df.iterrows():
        compound_polarity_score = sid.polarity_scores(row['text'])
        user_id = row['user_id']
        business_id = row['business_id']
        utility_matrix[business_id][user_id] = compound_polarity_score['compound']
    print(utility_matrix)


                       2aFiy99vNLklCx3T_tGS9A 2LfIuF3_sX6uwe-IR-P0jQ
KpkOkG6RIf4Ra25Lhhxf1A                 0.8895                    NaN
bQ7fQq1otn9hKX-gXRsrgA                 0.1513                    NaN
r1NUhdNmL6yU9Bn-Yx6FTw                  0.807                    NaN
aW3ix1KNZAvoM8q-WghA3Q                    NaN                 0.9523
YOo-Cip8HqvKp_p9nEGphw                    NaN                 0.6061
bgl3j8yJcRO-00NkUYsXGQ                    NaN                  0.944
CWKF9de-nskLYEqDDCfubg                    NaN                 0.8881
GJ7PTY7huYORFKKg3db3Gw                    NaN                 0.8945
rxqp9eXZj1jYTn0UIsm3Hg                    NaN                 0.8553
UU0nHQtHPMAfLidk8tOHTg                    NaN                 0.9426


In [124]:
#Print the first review
print(reviews[0])
print("\n",reviews[1])
print("\n",reviews[10000])

{"review_id":"NxL8SIC5yqOdnlXCg18IBg","user_id":"KpkOkG6RIf4Ra25Lhhxf1A","business_id":"2aFiy99vNLklCx3T_tGS9A","stars":5,"date":"2011-10-10","text":"If you enjoy service by someone who is as competent as he is personable, I would recommend Corey Kaplan highly. The time he has spent here has been very productive and working with him educational and enjoyable. I hope not to need him again (though this is highly unlikely) but knowing he is there if I do is very nice. By the way, I'm not from El Centro, CA. but Scottsdale, AZ.","useful":0,"funny":0,"cool":0,"type":"review"}

 {"review_id":"pXbbIgOXvLuTi_SPs1hQEQ","user_id":"bQ7fQq1otn9hKX-gXRsrgA","business_id":"2aFiy99vNLklCx3T_tGS9A","stars":5,"date":"2010-12-29","text":"After being on the phone with Verizon Wireless trying to figure out why my phone wasn't working for 4.5 hours, I was put in touch with Sharpie Tech. Well, after 10 seconds they fixed the problem. As the owner of a company that does our best numbers over the holiday seas

In [125]:
from nltk.tokenize import sent_tokenize
import json
first_review = json.loads(reviews[0])
second_review = json.loads(reviews[1])
nth_review = json.loads(reviews[10000])


In [126]:
review_text = first_review['text']
review_text2 = second_review['text']
review_text3 = nth_review['text']

In [127]:
# Sentiment Intensity Analyzer Using Vader
from __future__ import print_function
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize
reviews = [first_review, second_review, nth_review]
sid = SentimentIntensityAnalyzer()
for review in reviews:
    review_text = review['text']
    rating = review['stars']
    ss = sid.polarity_scores(review_text)
    print(review_text + "\n\nActual rating: {}".format(rating))
    for k in sorted(ss):
        print("{0}:{1} ".format(k, ss[k]),end='')
    print("\n\n")

If you enjoy service by someone who is as competent as he is personable, I would recommend Corey Kaplan highly. The time he has spent here has been very productive and working with him educational and enjoyable. I hope not to need him again (though this is highly unlikely) but knowing he is there if I do is very nice. By the way, I'm not from El Centro, CA. but Scottsdale, AZ.

Actual rating: 5
compound:0.8895 neg:0.0 neu:0.823 pos:0.177 


After being on the phone with Verizon Wireless trying to figure out why my phone wasn't working for 4.5 hours, I was put in touch with Sharpie Tech. Well, after 10 seconds they fixed the problem. As the owner of a company that does our best numbers over the holiday season, having my phone out of order for 4.5 hours was horrible. Sharpie Tech fixed the problem in record time, even Verizon was shocked. I highly recommend working with this company and I look forward to working with them more. 

-Rachel Charlupski
Founder and CEO, The Babysitting Compan